## Description

This notebook performs the experiments for nudge impacts for distributions generated by Ising models.

In [ ]:
import time
import os
import numpy as np
import networkx as nx
import pandas as pd
from r0bit import ising
import ising_model

### Generate the network used for the Ising model

#### Network settings

In [ ]:
number_of_nodes = 100
network_degree = 2

#### Generate the network

In [ ]:
network_backup = ising.generate_powerlaw_network(number_of_nodes, network_degree)
network = network_backup.copy()
ising_model.set_values_nodes_uniform(network)

### Gather samples to estimate the distributions by simulating the ising model

#### Ising model settings

In [ ]:
temperature = 2.5

#### simulation settings

In [ ]:
timesteps = 50
number_of_samples = 5
number_of_distributions = 1

#### perform the simulation and gather the samples and create the distributions

In [ ]:
start = time.time()

samples_folder = "ising_samples"
if not os.path.exists(samples_folder):
    os.makedirs(samples_folder)
    
for distribution_number in range(number_of_distributions):
    print("distribution number {}".format(distribution_number))
    network = ising.generate_powerlaw_network(number_of_nodes, network_degree)
    ising_model.set_values_nodes_uniform(network)
    nx.write_gpickle(network, "{}/network{}_network_size{}_network_degree{}_temp{:.1f}.pkl".format(
        samples_folder, distribution_number, number_of_nodes, network_degree, temperature
    ))
    samples = pd.DataFrame(
        data=np.zeros((number_of_samples, len(network.nodes()))), 
        index=range(number_of_samples), 
        columns=network.nodes()
    )
    for sample_number in range(number_of_samples):
        if sample_number%10 == 0 and sample_number != 0:
            print("sample number {}".format(sample_number))
        
        ising_model.update_network(network, timesteps, temperature)
        for node in network.nodes():
            samples.at[sample_number, node] = network.node[node]["value"]
            
    samples.to_pickle("{}/samples{}_network_size{}_network_degree{}_temp{:.1f}.pkl".format(
        samples_folder, distribution_number, number_of_nodes, network_degree, temperature
    ))
    #print(samples.loc[:, [1, 2]])
    
print("run time {}".format(time.time()-start))

### Convert the samples to distributions

#### settings for creating the distributions

In [ ]:
number_of_states = 2
max_number_of_neighbors = 8
distributions_folder = "ising_distributions"
if not os.path.exists(distributions_folder):
    os.makedirs(distributions_folder)


In [ ]:

for count in range(number_of_distributions):
    network = nx.read_gpickle("{}/network{}_network_size{}_network_degree{}_temp{:.1f}.pkl".format(
        samples_folder, count, number_of_nodes, network_degree, temperature
    ))
    samples = pd.read_pickle("{}/samples{}_network_size{}_network_degree{}_temp{:.1f}.pkl".format(
        samples_folder, count, number_of_nodes, network_degree, temperature
    ))
    selected_nodes = ising_model.select_nodes(network, max_number_of_neighbors)
    for selected_node in selected_nodes:
        print(list(network.neighbors(selected_node)))
        distribution = ising_model.convert_samples_to_dist(
            samples, selected_node, list(network.neighbors(selected_node)), number_of_states
        )
        print(distribution.shape)
        output_label = set([len(distribution.shape) - 1])
        input_labels = set(range(len(distribution.shape) - 1))
        marginal, conditional = ising_model.produce_marginal_and_conditional(
            distribution, input_labels, output_label
        )

        number_of_neighbors = len(list(network.neighbors(selected_node))) 
        filename_marginal = "{}/marginal{}_neighbors{}_network_size{}_network_degree{}_temp{:.2f}.npy".format(
            distributions_folder, count, number_of_neighbors, number_of_nodes, network_degree, temperature
        )
        filename_conditional = "{}/marginal{}_neighbors{}_network_size{}_network_degree{}_temp{:.2f}.npy".format(
            distributions_folder, count, number_of_neighbors, number_of_nodes, network_degree, temperature
        )
        np.save(filename_marginal, marginal)
        np.save(filename_conditional, conditional)
        #print(distribution)
        
        #input_distribution = 
        #conditional_output_distribution = 
        #save distributions



### Perform nudge experiments

In [ ]:

for count in range(number_of_distributions):
    for number_of_neighbors in range(1, max_number_of_neighbors):
        filename_marginal = "{}/marginal{}_neighbors{}_network_size{}_network_degree{}_temp{:.2f}.npy".format(
            distributions_folder, count, number_of_neighbors, number_of_nodes, network_degree, temperature
        )
        filename_conditional = "{}/marginal{}_neighbors{}_network_size{}_network_degree{}_temp{:.2f}.npy".format(
            distributions_folder, count, number_of_neighbors, number_of_nodes, network_degree, temperature
        )
        try:
            marginal = np.load(filename_marginal)
            conditional = np.load(filename_conditional)            
        except IOError:
            print("file not found")